In [1]:
import os, ssl
import requests
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

# Covid Cases per City in OC  

In [3]:
url = "https://ochca.maps.arcgis.com/apps/opsdashboard/index.html#/5839a554eaac45c2b8e05dacc74c3bec"
browser.visit(url)

In [4]:
html = browser.html
soup = bs(html,'html.parser')
data = soup.find_all("div", class_="external-html")

In [5]:
confirmed_case_city = []
for i in data:
    a = i.find("p").text
    b = a.split("-")
    city_name = b[0][:-1]
    confirmed_case = b[1].split(" ")[0][1:-6]
    if(confirmed_case):
        confirmed_case_city.append([city_name,confirmed_case])    

In [6]:
oc_case_df = pd.DataFrame(confirmed_case_city)

In [7]:
oc_case_df.columns = ["City", "Confirmed_case"]

In [8]:
oc_case_df.head()

,City,Confirmed_case
0,Aliso Viejo,194
1,Anaheim,4385
2,Brea,222
3,Buena Park,691
4,Costa Mesa,773


# Covid Cases per City in LA County¶

In [9]:
url = "http://publichealth.lacounty.gov/media/Coronavirus/locations.htm"
browser.visit(url)

In [10]:
html = browser.html
soup = bs(html,'html.parser')
data = soup.find_all("tr")

In [11]:
LA_case = []
for row in data:
    list_td = row.find_all("td")
    temp_list = []
    for item in list_td:
        if(item.text):
            temp_list.append(item.text)
    if(temp_list):
        if "City of " in temp_list[0]:
            city_data = temp_list[0][8:];
            temp_list[0] = city_data;
            LA_case.append(temp_list)

In [12]:
la_case_df = pd.DataFrame(LA_case)
la_case_df.columns= ["City", "Confirmed_case", "Case_Rate", "Deaths", "Death_Rate"]

In [13]:
la_case_df.head()

,City,Confirmed_case,Case_Rate,Deaths,Death_Rate
0,Agoura Hills,76,364,1,5
1,Alhambra,616,710,11,13
2,Arcadia,234,405,11,19
3,Artesia,163,971,3,18
4,Avalon,5,129,0,0


# Covid Cases per City in San Diego County

In [14]:
import tabula
from tabula import read_pdf

In [15]:
url = "https://www.sandiegocounty.gov/content/dam/sdc/hhsa/programs/phs/Epidemiology/COVID-19%20Daily%20Update_City%20of%20Residence.pdf"

In [16]:
df = read_pdf(url)

'pages' argument isn't specified.Will extract only from page 1 by default.


In [17]:
sandiego_df = df[0].drop([0])

In [18]:
sandiego_df.head()

,Unnamed: 0,"Summary of COVID-19 Cases by City of Residence, San Diego County"
1,Incorporated City,"17,580 87.5% 622.4"
2,Carlsbad,318 1.6% 277.4
3,Chula Vista,"2,665 13.3% 996.3"
4,Coronado,79 0.4% 364.3
5,Del Mar,24 0.1% ***


In [19]:
for index,row in sandiego_df.iterrows():
    split_data = row[1].split()
    length = len(split_data)
    count = split_data[0]
    if(length==3):
        percentage_total = split_data[1]
        rate_per_100000 = split_data[2]
    elif (length==2):
        percentage_total = split_data[1]
        rate_per_100000 = ""
    elif(length==1):
        percentage_total = ""
        rate_per_100000 = ""
    sandiego_df.loc[index,"count"] = count
    sandiego_df.loc[index,"percentage_total"] = percentage_total
    sandiego_df.loc[index,"rate_per_100000"] = rate_per_100000

In [20]:
sandiego_df.columns = ["City", "Summary", "Count", "Total_Rate", "Rate_per_10000"]

In [21]:
sandiego_df.drop(["Summary"], axis=1, inplace=True)

In [22]:
sandiego_df.head()

,City,Count,Total_Rate,Rate_per_10000
1,Incorporated City,"17,580",87.5%,622.4
2,Carlsbad,318,1.6%,277.4
3,Chula Vista,"2,665",13.3%,996.3
4,Coronado,79,0.4%,364.3
5,Del Mar,24,0.1%,***


# Covid Cases per City in Santa Barbara County

In [23]:
url = "https://publichealthsbc.org/status-reports/"
browser.visit(url)

In [24]:
html = browser.html
soup = bs(html,'html.parser')
data = soup.find_all("table")

In [25]:
a = data[0].find_all("tr")

In [26]:
sb_list = []
for i in a:
    j = i.find_all("td")
    temp_data = []
    for data in j:
        temp_data.append(data.text)
    sb_list.append(temp_data)


In [27]:
sb_df = pd.DataFrame(sb_list)

In [28]:
sb_df.columns = list(sb_df.iloc[0])

In [29]:
sb_df.drop([0],inplace=True)

In [30]:
sb_df

,Geographic Area as of 7/13/2020,DailyCases,Total Confirmed Cases,Recovered by Region,Active Cases by Region,Number of Deaths
1,SOUTH COUNTY UNINCORPORATED AREA includes comm...,1,77,69,7,0
2,CITY OF SANTA BARBARA and the unincorporated a...,4,471,417,52,3
3,CITY OF GOLETA,1,85,71,12,2
4,COMMUNITY OF ISLA VISTA,1,28,24,4,0
5,UNINCORPORATED AREA OF THE GOLETA VALLEY AND G...,2,68,61,6,1
6,SANTA YNEZ VALLEY including the Cities of Solv...,0,34,27,7,0
7,CITY OF LOMPOC and the communities of Mission ...,1,226,197,25,4
8,FEDERAL PRISON IN LOMPOC,0,998,994,1,3
9,CITY OF SANTA MARIA,38,1754,1570,167,18
10,COMMUNITY OF ORCUTT,2,123,108,14,0


# Covid Cases per City in Riverside County

In [31]:
import tabula
from tabula import read_pdf

In [32]:
url = "https://www.rivcoph.org/Portals/0/Documents/CoronaVirus/July/Dashboard/2/City_Report_071320.pdf?ver=2020-07-13-151405-373&timestamp=1594678461087"
df = read_pdf(url)

'pages' argument isn't specified.Will extract only from page 1 by default.


In [33]:
riverside_df = df[0]

In [34]:
riverside_df.head()

,Name,Count,Death,Percentage ** of Overall Total
0,County Jails,259,3,1.0%
1,State Prison,"1,156",1,4.5%
2,Incorporated City,"20,889",493,81.1%
3,Banning,284,10,1.1%
4,Beaumont,474,12,1.8%
